In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')


# 表示用の設定を変更します
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 130


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Load files

In [2]:
balance = pd.read_csv("./data/balance_sheet.csv")
income = pd.read_csv("./data/income.csv")
price = pd.read_csv("./data/price.csv")
market_news = pd.read_csv("./data/market_news.csv")
stock_news = pd.read_csv("./data/stock_news.csv")

price["date"] = pd.DatetimeIndex(price["label"])

income["date"] = pd.DatetimeIndex(pd.to_datetime(income['date'], unit="ms"))
balance["date"] = pd.DatetimeIndex(pd.to_datetime(balance['date'], unit="ms"))

market_news["date"] = pd.DatetimeIndex(market_news['date'])
stock_news["date"] = pd.DatetimeIndex(stock_news['date'])

market_news.rename(columns={'score':'market_news_score'}, inplace=True)
stock_news.rename(columns={'score':'stock_news_score'}, inplace=True)

# Drop date column because date of income and balance is same for each row
balance.drop(["date"], axis=1, inplace=True)

# Aggregate balance and income

In [3]:
balance_income = pd.merge(balance, income, how='left', on=['key', 'fiscalYear', 'fiscalQuarter'])

balance_income = balance_income.set_index("date", drop=True)


In [4]:
balance_income.reset_index(inplace=True)

for key in tqdm(balance_income["key"].unique()):
    balance_income.loc[balance_income[balance_income["key"] == key].index, "q_change_OperatingIncome"] = balance_income[balance_income["key"] == key][ "operatingIncome"].pct_change(-1)
    balance_income.loc[balance_income[balance_income["key"] == key].index, "shifted_operatingIncome"] = balance_income[balance_income["key"] == key][ "operatingIncome"].shift(-1)

    balance_income.loc[balance_income[balance_income["key"] == key].index, "q_change_NetIncome"] = balance_income[balance_income["key"] == key][ "netIncome"].pct_change(-1)
    balance_income.loc[balance_income[balance_income["key"] == key].index, "shifted_netIncome"] = balance_income[balance_income["key"] == key]["netIncome"].shift(-1)

    for quarter in balance_income[balance_income["key"] == key]["fiscalQuarter"].unique():
        balance_income.loc[balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)].index, "year_q_change_OperatingIncome"] = balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)]["operatingIncome"].pct_change(1)
        balance_income.loc[balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)].index, "year_q_change_NetIncome"] = balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)]["netIncome"].pct_change(1)

        balance_income.loc[balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)].index, "shifted_year_OperatingIncome"] = balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)]["operatingIncome"].shift(1)
        balance_income.loc[balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)].index, "shifted_year_NetIncome"] = balance_income[(balance_income["key"] == key) & (balance_income["fiscalQuarter"] == quarter)]["netIncome"].shift(1)

balance_income.loc[balance_income[(balance_income["shifted_operatingIncome"] < 0) & (balance_income["operatingIncome"] > 0)].index, "q_change_OperatingIncome"] = -balance_income[(balance_income["shifted_operatingIncome"] < 0) & (balance_income["operatingIncome"] > 0)]["q_change_OperatingIncome"]
balance_income.loc[balance_income[(balance_income["shifted_netIncome"] < 0) & (balance_income["netIncome"] > 0)].index, "q_change_NetIncome"] = -balance_income[(balance_income["shifted_netIncome"] < 0) & (balance_income["netIncome"] > 0)]["q_change_NetIncome"]

balance_income.loc[balance_income[(balance_income["shifted_year_OperatingIncome"] < 0) & (balance_income["operatingIncome"] > 0)].index, "year_q_change_OperatingIncome"] = -balance_income[(balance_income["shifted_year_OperatingIncome"] < 0) & (balance_income["operatingIncome"] > 0)]["year_q_change_OperatingIncome"]
balance_income.loc[balance_income[(balance_income["shifted_year_NetIncome"] < 0) & (balance_income["netIncome"] > 0)].index, "year_q_change_NetIncome"] = -balance_income[(balance_income["shifted_year_NetIncome"] < 0) & (balance_income["netIncome"] > 0)]["year_q_change_NetIncome"]


fill_cols = ["year_q_change_OperatingIncome",  "q_change_OperatingIncome", "q_change_NetIncome", "year_q_change_NetIncome"]
balance_income[fill_cols] = balance_income[fill_cols].fillna(0)


balance_income.drop(["shifted_netIncome", "shifted_operatingIncome", "shifted_year_OperatingIncome", "shifted_year_NetIncome"], axis=1, inplace=True)

balance_income.set_index("date", drop=True, inplace=True)


  0%|          | 0/10 [00:00<?, ?it/s]

# Create balance sheet and income info for all dates

We need to do this to correctly aggregate balance sheet and income data with price data because price data doesn't have data for Saturdays and Sundays; therefore, we need to create balance sheet and income statement for all business days as well.

In [5]:
# This is the unction to upsample data.
# For example, if there are two balance sheets of Apple for dates "2020-01-15" and 2020-04-15" ,
# it will fill create data of 2020-01-15, 2020-01-16, ... , 2020-04-14, 2020-04-15 filling values with "2020-01-15" balance sheet data.
def upsample(df, key):
    df = df[df["key"] == key]
    df = df.resample('D', convention='start').asfreq()
    df = df.fillna(method="ffill")
    
    return df

In [6]:
keys = balance["key"].unique()

temp = []

for key in tqdm(keys):
    sample = upsample(balance_income, key)
    temp.append(sample)
    
balance_income = pd.concat(temp)

balance_income.reset_index(inplace=True)

balance_income[balance_income["key"] == "AAPL"].head()

  0%|          | 0/10 [00:00<?, ?it/s]

,date,accountsPayable,capitalSurplus,commonStock,currency_x,currentAssets,currentCash,currentLongTermDebt,filingType_x,fiscalDate_x,fiscalQuarter,fiscalYear,goodwill,intangibleAssets,inventory,longTermDebt,longTermInvestments,minorityInterest_x,netTangibleAssets,otherAssets,otherCurrentAssets,otherCurrentLiabilities,otherLiabilities,propertyPlantEquipment,receivables,reportDate_x,retainedEarnings,shareholderEquity,shortTermInvestments,symbol_x,totalAssets,totalCurrentLiabilities,totalLiabilities,treasuryStock,id_x,key,subkey_x,updated_x,costOfRevenue,currency_y,ebit,filingType_y,fiscalDate_y,grossProfit,incomeTax,interestIncome,minorityInterest_y,netIncome,netIncomeBasic,operatingExpense,operatingIncome,otherIncomeExpenseNet,pretaxIncome,reportDate_y,researchAndDevelopment,sellingGeneralAndAdmin,symbol_y,totalRevenue,id_y,subkey_y,updated_y,q_change_OperatingIncome,q_change_NetIncome,year_q_change_OperatingIncome,year_q_change_NetIncome
0,2015-03-28,2.315900e+10,NaN,2.317520e+10,USD,6.789100e+10,3.309600e+10,3.799000e+09,10-Q,2015-03-28,2.0,2015.0,4.711000e+09,4.061000e+09,2.396000e+09,4.007200e+10,1.933030e+11,0.0,1.202340e+11,3.937000e+09,9.094000e+09,3.177100e+10,9.562000e+09,2.015100e+10,1.816400e+10,2015-04-28,1.009200e+11,1.290060e+11,9.094000e+09,AAPL,2.611940e+11,5.872900e+10,1.321880e+11,0.0,BALANCE_SHEET,AAPL,quarterly,1.612292e+12,3.435400e+10,USD,1.827800e+10,10-Q,2015-03-28,2.365600e+10,4.995000e+09,0.0,0.0,1.356900e+10,1.356900e+10,3.973200e+10,1.827800e+10,0.0,1.856400e+10,2015-04-28,1.918000e+09,3.460000e+09,AAPL,5.801000e+10,INCOME,quarterly,1.608331e+12,0.0,0.0,0.306785,0.29032
1,2015-03-29,2.315900e+10,NaN,2.317520e+10,USD,6.789100e+10,3.309600e+10,3.799000e+09,10-Q,2015-03-28,2.0,2015.0,4.711000e+09,4.061000e+09,2.396000e+09,4.007200e+10,1.933030e+11,0.0,1.202340e+11,3.937000e+09,9.094000e+09,3.177100e+10,9.562000e+09,2.015100e+10,1.816400e+10,2015-04-28,1.009200e+11,1.290060e+11,9.094000e+09,AAPL,2.611940e+11,5.872900e+10,1.321880e+11,0.0,BALANCE_SHEET,AAPL,quarterly,1.612292e+12,3.435400e+10,USD,1.827800e+10,10-Q,2015-03-28,2.365600e+10,4.995000e+09,0.0,0.0,1.356900e+10,1.356900e+10,3.973200e+10,1.827800e+10,0.0,1.856400e+10,2015-04-28,1.918000e+09,3.460000e+09,AAPL,5.801000e+10,INCOME,quarterly,1.608331e+12,0.0,0.0,0.306785,0.29032
2,2015-03-30,2.315900e+10,NaN,2.317520e+10,USD,6.789100e+10,3.309600e+10,3.799000e+09,10-Q,2015-03-28,2.0,2015.0,4.711000e+09,4.061000e+09,2.396000e+09,4.007200e+10,1.933030e+11,0.0,1.202340e+11,3.937000e+09,9.094000e+09,3.177100e+10,9.562000e+09,2.015100e+10,1.816400e+10,2015-04-28,1.009200e+11,1.290060e+11,9.094000e+09,AAPL,2.611940e+11,5.872900e+10,1.321880e+11,0.0,BALANCE_SHEET,AAPL,quarterly,1.612292e+12,3.435400e+10,USD,1.827800e+10,10-Q,2015-03-28,2.365600e+10,4.995000e+09,0.0,0.0,1.356900e+10,1.356900e+10,3.973200e+10,1.827800e+10,0.0,1.856400e+10,2015-04-28,1.918000e+09,3.460000e+09,AAPL,5.801000e+10,INCOME,quarterly,1.608331e+12,0.0,0.0,0.306785,0.29032
3,2015-03-31,2.315900e+10,NaN,2.317520e+10,USD,6.789100e+10,3.309600e+10,3.799000e+09,10-Q,2015-03-28,2.0,2015.0,4.711000e+09,4.061000e+09,2.396000e+09,4.007200e+10,1.933030e+11,0.0,1.202340e+11,3.937000e+09,9.094000e+09,3.177100e+10,9.562000e+09,2.015100e+10,1.816400e+10,2015-04-28,1.009200e+11,1.290060e+11,9.094000e+09,AAPL,2.611940e+11,5.872900e+10,1.321880e+11,0.0,BALANCE_SHEET,AAPL,quarterly,1.612292e+12,3.435400e+10,USD,1.827800e+10,10-Q,2015-03-28,2.365600e+10,4.995000e+09,0.0,0.0,1.356900e+10,1.356900e+10,3.973200e+10,1.827800e+10,0.0,1.856400e+10,2015-04-28,1.918000e+09,3.460000e+09,AAPL,5.801000e+10,INCOME,quarterly,1.608331e+12,0.0,0.0,0.306785,0.29032
4,2015-04-01,2.315900e+10,NaN,2.317520e+10,USD,6.789100e+10,3.309600e+10,3.799000e+09,10-Q,2015-03-28,2.0,2015.0,4.711000e+09,4.061000e+09,2.396000e+09,4.007200e+10,1.933030e+11,0.0,1.202340e+11,3.937000e+09,9.094000e+09,3.177100e+10,9.562000e+09,2.015100e+10,1.816400e+10,2015-04-28,1.009200e+11,1.290060e+11,9.094000e+09,AAPL,2.611940e+11,5.872900e+10,1.32188

# Aggregate balance_income and stock price data

In [7]:
fin_price = price.merge(balance_income, how="left", on=["key", "date"])

fin_price.tail(3)


,change,changeOverTime,changePercent,close,fClose,fHigh,fLow,fOpen,fVolume,high,id,key,label,low,marketChangeOverTime,open,subkey,symbol,uClose,uHigh,uLow,uOpen,uVolume,updated,volume,date,accountsPayable,capitalSurplus,commonStock,currency_x,currentAssets,currentCash,currentLongTermDebt,filingType_x,fiscalDate_x,fiscalQuarter,fiscalYear,goodwill,intangibleAssets,inventory,longTermDebt,longTermInvestments,minorityInterest_x,netTangibleAssets,otherAssets,otherCurrentAssets,otherCurrentLiabilities,otherLiabilities,propertyPlantEquipment,receivables,reportDate_x,retainedEarnings,shareholderEquity,shortTermInvestments,symbol_x,totalAssets,totalCurrentLiabilities,totalLiabilities,treasuryStock,id_x,subkey_x,updated_x,costOfRevenue,currency_y,ebit,filingType_y,fiscalDate_y,grossProfit,incomeTax,interestIncome,minorityInterest_y,netIncome,netIncomeBasic,operatingExpense,operatingIncome,otherIncomeExpenseNet,pretaxIncome,reportDate_y,researchAndDevelopment,sellingGeneralAndAdmin,symbol_y,totalRevenue,id_y,subkey_y,updated_y,q_change_OperatingIncome,q_change_NetIncome,year_q_change_OperatingIncome,year_q_change_NetIncome
12585,0.66,-0.772560,0.0121,55.12,50.5435,50.6168,49.7090,49.8466,21188682,55.20,HISTORICAL_PRICES,MSFT,"Apr 6, 16",54.21,-0.772560,54.36,NaN,MSFT,55.12,55.20,54.21,54.36,21188682,2021-02-17 04:18:44,21188682,2016-04-06,6.759000e+09,NaN,7.895000e+09,USD,1.284210e+11,1.055520e+11,5.498000e+09,10-Q,2016-03-31,3.0,2016.0,1.794800e+10,4.459000e+09,2.450000e+09,4.089600e+10,5.344800e+10,0.0,5.239900e+10,2.895000e+09,6.598000e+09,3.209700e+10,1.913900e+10,1.683100e+10,1.224700e+10,2016-04-21,4.954000e+09,7.480600e+10,6.598000e+09,MSFT,1.818690e+11,4.435400e+10,1.070630e+11,0.0,BALANCE_SHEET,quarterly,1.611842e+12,7.722000e+09,USD,5.283000e+09,10-Q,2016-03-31,1.280900e+10,1.280000e+09,340000000.0,0.0,3.756000e+09,3.756000e+09,1.524800e+10,5.283000e+09,0.0,5.036000e+09,2016-04-21,2.980000e+09,4.546000e+09,MSFT,2.053100e+10,INCOME,quarterly,1.611948e+12,-0.123299,-0.248499,-0.055595,-0.217663
12586,-0.56,-0.774871,-0.0102,54.56,50.0300,50.7085,49.9383,50.6077,19272255,55.30,HISTORICAL_PRICES,MSFT,"Apr 5, 16",54.46,-0.774871,55.19,NaN,MSFT,54.56,55.30,54.46,55.19,19272255,2021-02-17 04:17:30,19272255,2016-04-05,6.759000e+09,NaN,7.895000e+09,USD,1.284210e+11,1.055520e+11,5.498000e+09,10-Q,2016-03-31,3.0,2016.0,1.794800e+10,4.459000e+09,2.450000e+09,4.089600e+10,5.344800e+10,0.0,5.239900e+10,2.895000e+09,6.598000e+09,3.209700e+10,1.913900e+10,1.683100e+10,1.224700e+10,2016-04-21,4.954000e+09,7.480600e+10,6.598000e+09,MSFT,1.818690e+11,4.435400e+10,1.070630e+11,0.0,BALANCE_SHEET,quarterly,1.611842e+12,7.722000e+09,USD,5.283000e+09,10-Q,2016-03-31,1.280900e+10,1.280000e+09,340000000.0,0.0,3.756000e+09,3.756000e+09,1.524800e+10,5.283000e+09,0.0,5.036000e+09,2016-04-21,2.980000e+09,4.546000e+09,MSFT,2.053100e+10,INCOME,quarterly,1.611948e+12,-0.123299,-0.248499,-0.055595,-0.217663
12587,0.87,-0.771281,0.0159,55.43,50.8277,51.0387,50.4335,50.8277,18928810,55.66,HISTORICAL_PRICES,MSFT,"Apr 4, 16",55.00,-0.771281,55.43,NaN,MSFT,55.43,55.66,55.00,55.43,18928810,2021-02-17 04:17:36,18928810,2016-04-04,6.759000e+09,NaN,7.895000e+09,USD,1.284210e+11,1.055520e+11,5.498000e+09,10-Q,2016-03-31,3.0,2016.0,1.794800e+10,4.459000e+09,2.450000e+09,4.089600e+10,5.344800e+10,0.0,5.239900e+10,2.895000e+09,6.598000e+09,3.209700e+10,1.913900e+10,1.683100e+10,1.224700e+10,2016-04-21,4.954000e+09,7.480600e+10,6.598000e+09,MSFT,1.818690e+11,4.435400e+10,1.070630e+11,0.0,BALANCE_SHEET,quarterly,1.611842e+12,7.722000e+09,USD,5.283000e+09,10-Q,2016-03-31,1.280900e+10,1.280000e+09,340000000.0,0.0,3.756000e+09,3.756000e+09,1.524800e+10,5.283000e+09,0.0,5.036000e+09,2016-04-21,2.980000e+09,4.546000e+09,MSFT,2.053100e+10,INCOME,quarterly,1.611948e+12,-0.123299,-0.248499,-0.055595,-0.217663


# Check if the data is correctly aggregated

In [8]:
print("Aggregated balance sheet and income")
balance_income[(balance_income["date"] > "2019-12-25") & (balance_income["date"] < "2019-12-30") & (balance_income["key"] == "AAPL")][["key", "fiscalYear", "fiscalQuarter", "date"]]


print("\n\n Aggregated balance sheet, income, and price")
fin_price[(fin_price["date"] > "2019-12-25") & (fin_price["date"] < "2020-01-01") & (fin_price["key"] == "AAPL")][["key", "fiscalYear", "fiscalQuarter", "date"]].sort_values(by="date", ascending=True)


Aggregated balance sheet and income


,key,fiscalYear,fiscalQuarter,date
1734,AAPL,2019.0,4.0,2019-12-26
1735,AAPL,2019.0,4.0,2019-12-27
1736,AAPL,2020.0,1.0,2019-12-28
1737,AAPL,2020.0,1.0,2019-12-29




 Aggregated balance sheet, income, and price


,key,fiscalYear,fiscalQuarter,date
318,AAPL,2019.0,4.0,2019-12-26
317,AAPL,2019.0,4.0,2019-12-27
316,AAPL,2020.0,1.0,2019-12-30
315,AAPL,2020.0,1.0,2019-12-31


# Aggregate news and market news data

Get mean score from market news at that day and mean score from stock news at that day and company.

In [9]:
market_mean_score = market_news.groupby("date")["market_news_score"].mean().reset_index()
stock_mean_score = stock_news.groupby(["date", "ticker"])["stock_news_score"].mean().reset_index()

Aggregate them

In [10]:
fin_price_news = pd.merge(fin_price, market_mean_score, how='left', on=['date'])
fin_price_news = pd.merge(fin_price_news, stock_mean_score, how='left', left_on=['key','date'], right_on=["ticker", "date"])

Check whether stock news was correctly aggregated

In [11]:
fin_price_news[(fin_price_news["date"] > "2020-04-01") & (fin_price_news["date"] < "2020-04-08") & (fin_price_news["key"] == "AAPL")][["key", "fiscalYear", "fiscalQuarter", "date", "stock_news_score"]]
fin_price_news[(fin_price_news["date"] > "2020-05-26") & (fin_price_news["date"] < "2020-05-30") & (fin_price_news["key"] == "AAPL")][["key", "fiscalYear", "fiscalQuarter", "date", "stock_news_score"]]

stock_news.head(2)

,key,fiscalYear,fiscalQuarter,date,stock_news_score
248,AAPL,2020.0,2.0,2020-04-07,NaN
249,AAPL,2020.0,2.0,2020-04-06,-0.128
250,AAPL,2020.0,2.0,2020-04-03,NaN
251,AAPL,2020.0,2.0,2020-04-02,NaN


,key,fiscalYear,fiscalQuarter,date,stock_news_score
212,AAPL,2020.0,2.0,2020-05-29,NaN
213,AAPL,2020.0,2.0,2020-05-28,0.4019
214,AAPL,2020.0,2.0,2020-05-27,NaN


,ticker,headline,source,url,date,stock_news_score
0,AAPL,Dow Futures Soar as New York Begins Winning th...,ccn.com,https://www.ccn.com/dow-futures-soar-as-new-yo...,2020-04-06,-0.1280
1,AAPL,Apple Snaps Up AI Startup Inductiv As Analysts...,smarteranalyst.com,https://www.smarteranalyst.com/yahoo/apple-sna...,2020-05-28,0.4019


Check whether market news was correctly aggregated

In [12]:
fin_price_news[(fin_price_news["date"] > "2020-03-31") & (fin_price_news["date"] < "2020-04-07") & (fin_price_news["key"] == "AAPL")][["key", "fiscalYear", "fiscalQuarter", "date", "market_news_score"]]

market_mean_score.head().sort_values(by="date", ascending=False)

,key,fiscalYear,fiscalQuarter,date,market_news_score
249,AAPL,2020.0,2.0,2020-04-06,-0.200040
250,AAPL,2020.0,2.0,2020-04-03,0.000000
251,AAPL,2020.0,2.0,2020-04-02,0.001914
252,AAPL,2020.0,2.0,2020-04-01,0.050200


,date,market_news_score
4,2020-04-06,-0.200040
3,2020-04-05,0.000000
2,2020-04-03,0.000000
1,2020-04-02,0.001914
0,2020-04-01,0.050200


In [13]:
fin_price.to_csv("./data/fin_price.csv", index=False)
fin_price_news.to_csv("./data/fin_price_news.csv", index=False)
